import libraries

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

ImageDataGenerator for training data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
)

Training and validation data from directories

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory='techosmotr/train',
    target_size=(250, 250),
    batch_size=32,
    seed=42,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory='techosmotr/train',
    target_size=(250, 250),
    batch_size=32,
    seed=42,
    class_mode='categorical',
    subset='validation'
)

Build the model

In [ ]:
model = Sequential()

model.add(Conv2D(input_shape=(250, 250, 3), filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

Model summary

In [ ]:
print(model.summary())

Compile the model

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Train the model

In [ ]:
model_fit = model.fit(
    train_generator,
    epochs=12,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
)

Save the model

In [ ]:
model.save('model3')